### CIS 4360 Computational Methods in Finance

In [16]:
import math
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
from scipy.stats import skew, kurtosis
from math import log, exp, sqrt, pow

from stock import Stock
import option
import os
import sqlite3
from financial_option import *
import datetime

**Property of normal distribution**

$$ ln S_T \sim \phi \left[ ln S_0 + \left(\mu - \frac{\sigma^2}{2}\right) T, \sigma^2 T\right]$$

**There is a 95% probability that a normal variable will fall within 1.96 standard deviations**

**The 95% confidence interval of the normal variable will be between (mean - 1.96 stdev) and (mean+1.96 stdev)**  

## Example 15.1

**Consider a stock with an initial price of 40, an expected return of 16\% per annum, and a volatility of 20%. What is the 95% confidence interval of the stock price in 6 months?**

In [2]:
# Example 15.1
S0 = 40
mu = 0.16
sigma = 0.2
T = 6/12

In [3]:
mean = log(S0) + (mu - pow(sigma, 2)/2)*T
stdev = sigma * sqrt(T)
print(mean, stdev)

3.758879454113936 0.14142135623730953


In [4]:
lower_bound = exp(mean - 1.96*stdev)
upper_bound = exp(mean + 1.96*stdev)
print(lower_bound, upper_bound)

32.51474230558975 56.603188205344175


## Example 15.6

**The stock price 6 months from the expiration of an option is 42, exercise price of the option is 40, risk-free rate is 10% and volatility is 20%. What should be the call and put priced at?**

In [5]:
from scipy.stats import norm

In [6]:
S0 = 42
K = 40
r = 0.1
sigma = 0.2
T = 0.5

In [7]:
d1 = (log(S0/K)+(r + sigma**2/2)*T)/(sigma * sqrt(T))
d2 = (log(S0/K)+(r - sigma**2/2)*T)/(sigma * sqrt(T))
print(d1, d2)

0.7692626281060315 0.6278412718687221


In [8]:
c = S0* norm.cdf(d1) - K*exp(-r*T)*norm.cdf(d2)
print(c)

4.759422392871532


In [9]:
p = K * exp(-r*T)*norm.cdf(-d2) - S0*norm.cdf(-d1)

In [10]:
p

0.8085993729000922

### Now let's compare that with your Black-Schole Model Implementation

***That is, use your OO Black-Scholes model implementation to calculate the price of the above options. Make sure they reproduce the same answer***

***Import your library***

In [25]:
from blackscholes_model import BlackScholesModel

***Question 1: First create a Stock object with the spot_price and the sigma***

In [22]:
opt = option.Option()
opt.data_dir = "./data"
opt.output_dir = os.path.join(opt.data_dir, "daily")
opt.sqlite_db = os.path.join(opt.data_dir, "sqlitedb/Equity.db")
db_file = opt.sqlite_db
db_connection = sqlite3.connect(db_file)

S0 = 42 #spot
sigma = .2 #variance

#Stock Obj
stock = Stock(opt, db_connection, 'AAPL', freq='quarterly', spot_price=S0, sigma=sigma)

***Question 2: Create instance of the model***

In [23]:
pricing_date = datetime.date(2023, 12,8)
r = 0.1 #risk-free-rate
bs_model = BlackScholesModel(pricing_date, r)

***Question 3: Now call your model calc_model_price to calculate the above options***

In [27]:
K = 40 #strike
T = 0.5 #time-to-expiry

option_call = EuropeanCallOption(stock, time_to_expiry=T, strike=K)
option_put = EuropeanPutOption(stock, time_to_expiry=T, strike=K)

option_call_price = bs_model.calc_model_price(option_call)
option_put_price = bs_model.calc_model_price(option_put)

print('Call Option Price: ', option_call_price)
print('Put Option Price: ', option_put_price)

Call Option Price:  4.759422392871532
Put Option Price:  0.8085993729000922


***Question 4: Now call your model to calculate ALL the greeks for both options***

In [28]:
# calc delta, gamma, theta, vega for both the call and the put

#Call for greeks
call_delta = bs_model.calc_delta(option_call)
call_gamma = bs_model.calc_gamma(option_call)
call_theta = bs_model.calc_theta(option_call)
call_vega = bs_model.calc_vega(option_call)
call_rho = bs_model.calc_rho(option_call)
#Put for greeks
put_delta = bs_model.calc_delta(option_put)
put_gamma = bs_model.calc_gamma(option_put)
put_theta = bs_model.calc_theta(option_put)
put_vega = bs_model.calc_vega(option_put)
put_rho = bs_model.calc_rho(option_put)

print('Call Delta:', call_delta)
print('Call Gamma:', call_gamma)
print('Call Theta:', call_theta)
print('Call Vega:', call_vega)
print('Call Rho:', call_rho)

print('Put Delta:', put_delta)
print('Put Gamma:', put_gamma)
print('Put Theta:', put_theta)
print('Put Vega:', put_vega)
print('Put Rho:', put_rho)

Call Delta: 0.779131290942669
Call Gamma: 0.04996267040591185
Call Theta: -4.559092194592627
Call Vega: 8.813415059602853
Call Rho: 13.982045913360283
Put Delta: -0.22086870905733103
Put Gamma: 0.04996267040591185
Put Theta: 1.0584711857367115
Put Vega: 8.813415059602853
Put Rho: -5.0425425766539975
